In [7]:
from torch import tensor
import torch
import numpy as np

Debugging the old xcdiff's atomization energies code

In [49]:
def atomization_energies(energies):
    """[summary]

    Args:
        energies ([type]): [description]
    """
    def split(el):
        """[summary]

        Args:
            el ([type]): [description]

        Returns:
            [type]: [description]
        """
        import re
        res_list = [s for s in re.split("([A-Z][^A-Z]*)", el) if s]
        return res_list


    ae = {}
    for key in energies:
        if isinstance(energies[key],torch.Tensor):
            if len(split(key)) == 1:continue
            e_tot = energies[key].clone()
            e_tot_key = key
        else:
            e_tot = np.array(energies[key])
            e_tot_key = key
        print('AE Loss Call: Key/Energy Start ', key, e_tot)
        for symbol in split(key):
            e_tot -= energies[symbol]
            print('{} - {}: {} - {} = {}'.format(e_tot_key, symbol, e_tot + energies[symbol], energies[symbol], e_tot))
        ae[key] = e_tot
        print('AE Loss Result: ', ae)
    return ae

def atomization_energies2(energies):
    """Calculates atomization energies based on a dictionary of molecule/atomic energies.
    
    energies['ABCD'] = molecular energy
    energies['A'], energies['B'], etc. = atomic energy.
    
    Loops over ABCD - A - B - C - D

    Args:
        energies (dict): dictionary of molecule and constituent atomic energies.
    """
    def split(el):
        """Regex split molecule's symbolic expansion into constituent elements.
        No numbers must be present -- CH2 = CHH.

        Args:
            el (str): Molecule symbols

        Returns:
            list: list of individual atoms in molecule
        """
        import re
        res_list = [s for s in re.split("([A-Z][^A-Z]*)", el) if s]
        return res_list


    ae = {}
    for key in energies:
        if isinstance(energies[key],torch.Tensor):
            #if len(split(key)) == 1:continue
            e_tot = torch.clone(energies[key])
            e_tot_size = e_tot.size()
        else:
            e_tot = np.array(energies[key])
            e_tot_size = e_tot.shape
        symsplit = split(key)
        #If it is an AA reaction, energy difference between same species configurations.
        #Don't subtract A off twice, subtract AA-A for the target energy difference.
        if symsplit == ['A', 'A']:
            symsplit = ['A']
        for symbol in symsplit:
            #if single atom, continue
            if len(split(key)) == 1: continue
            e_sub = energies[symbol]
            e_sub_size = e_sub.size() if isinstance(e_sub, torch.Tensor) else e_sub.shape
            if e_tot_size == e_sub_size:
                e_tot -= e_sub
            else:
                e_tot -= e_sub[-1:]
            print('{} - {}: {}'.format(key, symbol, e_tot))
            ae[key] = e_tot
    if ae == {}:
        #empty dict -- no splitting occurred, so single atom
        ae[key] = e_tot
    print("Atomization Energy Final")
    print(ae)
    return ae


Must work on a molecule for atomization energies, A+B->AB reactions, and A+A-> AA reactions

In [50]:
test1 = {'A': tensor([-0.5000]), 'B': tensor([-184.6712]), 'AB': tensor([-185.1557])}
test1ref = .0289
test2 = {'A': tensor([-18.4576]), 'AA':tensor([-22.8865*2])}
test2ref = -0.0618
test3 = {'COO': tensor([-27.6939, -27.5877, -27.5128, -27.4599, -27.4224, -27.3957, -27.3767, -27.3630, -27.3532, -27.3461]),
         'C': tensor([-11.3933]), 'O': tensor([-17.8538])}
test3ref = -.6223
test4 = {'A': tensor([-0.5000]), 'B': tensor([-184.6712]), 'AB': tensor([-185.1557])}
test4ref = 0.0289

In [51]:
print(atomization_energies(test1), test1ref)
print(atomization_energies(test2), test2ref)
print(atomization_energies(test3), test3ref)
print(atomization_energies(test4), test4ref)

AE Loss Call: Key/Energy Start  AB tensor([-185.1557])
AB - A: tensor([-185.1557]) - tensor([-0.5000]) = tensor([-184.6557])
AB - B: tensor([-184.6557]) - tensor([-184.6712]) = tensor([0.0155])
AE Loss Result:  {'AB': tensor([0.0155])}
{'AB': tensor([0.0155])} 0.0289
AE Loss Call: Key/Energy Start  AA tensor([-45.7730])
AA - A: tensor([-45.7730]) - tensor([-18.4576]) = tensor([-27.3154])
AA - A: tensor([-27.3154]) - tensor([-18.4576]) = tensor([-8.8578])
AE Loss Result:  {'AA': tensor([-8.8578])}
{'AA': tensor([-8.8578])} -0.0618
AE Loss Call: Key/Energy Start  COO tensor([-27.6939, -27.5877, -27.5128, -27.4599, -27.4224, -27.3957, -27.3767,
        -27.3630, -27.3532, -27.3461])
COO - C: tensor([-27.6939, -27.5877, -27.5128, -27.4599, -27.4224, -27.3957, -27.3767,
        -27.3630, -27.3532, -27.3461]) - tensor([-11.3933]) = tensor([-16.3006, -16.1944, -16.1195, -16.0666, -16.0291, -16.0024, -15.9834,
        -15.9697, -15.9599, -15.9528])
COO - O: tensor([-16.3006, -16.1944, -16.1195

In [43]:
print(atomization_energies2(test1), test1ref)
test2a = {'A': tensor([-18.4576]), 'AA':tensor([-22.8865])}
print(atomization_energies2(test2a), test2ref)
print(atomization_energies2(test3), test3ref)

AB - A: tensor([-184.6557])
AB - B: tensor([0.0155])
Atomization Energy Final
{'AB': tensor([0.0155])}
{'AB': tensor([0.0155])} 0.0289
AA - A: tensor([-4.4289])
Atomization Energy Final
{'AA': tensor([-4.4289])}
{'AA': tensor([-4.4289])} -0.0618
COO - C: tensor([-16.3006, -16.1944, -16.1195, -16.0666, -16.0291, -16.0024, -15.9834,
        -15.9697, -15.9599, -15.9528])
COO - O: tensor([1.5532, 1.6594, 1.7343, 1.7872, 1.8247, 1.8514, 1.8704, 1.8841, 1.8939,
        1.9010])
COO - O: tensor([19.4070, 19.5132, 19.5881, 19.6410, 19.6785, 19.7052, 19.7242, 19.7379,
        19.7477, 19.7548])
Atomization Energy Final
{'COO': tensor([19.4070, 19.5132, 19.5881, 19.6410, 19.6785, 19.7052, 19.7242, 19.7379,
        19.7477, 19.7548])}
{'COO': tensor([19.4070, 19.5132, 19.5881, 19.6410, 19.6785, 19.7052, 19.7242, 19.7379,
        19.7477, 19.7548])} -0.6223
